# Exploratory data analysis of the dutch road accidents 
_Timespan of data: 2004-2015_

<img src='./img/accident.jpeg'>

Author: Erfan Nariman  
Date: 20-09-2020

In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

from src.get_files import GetFiles

### Check if csv's are in the data folder, else download them dutch government [website](https://data.overheid.nl/dataset/verkeersongevallen)

In [2]:
get_files = GetFiles()

if len(os.listdir("./data")) == 0:
    get_files.download_csvs()
else:
    print("CSV's of accidents are already downloaded and in the data folder")

CSV's of accidents are already downloaded and in the data folder


### Get all files with the `.csv` suffix

In [3]:
files = sorted([file for file in os.listdir("./data") if file.endswith(".csv")])
files

['ongevallen-2004.csv',
 'ongevallen-2005.csv',
 'ongevallen-2006.csv',
 'ongevallen-2007.csv',
 'ongevallen-2008.csv',
 'ongevallen-2009.csv',
 'ongevallen-2010.csv',
 'ongevallen-2011.csv',
 'ongevallen-2012.csv',
 'ongevallen-2013.csv',
 'ongevallen-2014.csv',
 'ongevallen-2015.csv']

### Read in the data

While reading in the csv's, we assign a new column called `Year` which is the number extracted from the filename

In [4]:
# load list of columns to keep while reading in data
from src.mapping import use_cols

dfs = []
for file in files:
    df = pd.read_csv(f"./data/{file}", low_memory=False)
    if "Volgnummer_Partijen" in df.columns:
        df = df.rename(columns={"Volgnummer_Partijen": "Volgnummer", "Provincie": "ProvincieNaam"})
    if "Latitidue" in df.columns:
        df = df.rename(columns={"Latitidue": "Latitude"})
    df = df[use_cols]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.shape

(1326104, 87)

### Remove columns with more than `80%` missing values

1. get sum of `NaN` per column and sort them descending
2. get the column names which have more than 80% missing values
3. drop these columns

In [5]:
# 1
nan = df.isna().sum().sort_values(ascending=False)
# 2
to_remove = nan[nan.div(df.shape[0]).gt(0.8)].index
print(to_remove)
# 3
df = df.drop(columns=to_remove)
df.shape

Index(['AantalPassagiers', 'BijzonderheidTypeInfrastructuur3',
       'VastgelegdePlaats2', 'BijzonderheidTidelijkAard3',
       'GevaarlijkStoffenPlaat', 'WegsituatieAnders', 'Zichtafstand',
       'BijzonderheidTypeInfrastructuur2', 'Beweging_An',
       'BijzonderheidTidelijkAard2', 'BijzonderheidTidelijkAard_An',
       'Weergesteldheid2', 'BijzonderheidTypeVerkeersmaatregel3',
       'Wegdekken_Anders', 'Beweging2', 'Aangrijppunt2',
       'Wegverharding_Anders', 'RibewijsBeginnr', 'VastgelegdePlaats_An',
       'AanhangenWagen', 'BromfietsCertificate', 'HectoLetter',
       'BijzonderheidTypeVerkeersmaatregel_An', 'ObjectType_Anders',
       'Wegvak_EndDat', 'BijzonderheidTypeInfrastructuur_An',
       'BijzonderheidTidelijkAard1', 'BijzonderheidTypeVerkeersmaatregel2',
       'BijzonderheidTypeInfrastructuur1', 'Lengte', 'WijkNaam', 'Breedte',
       'Doorrijder', 'SamengesteldLocatieNummer_Wegvak',
       'SamengesteldLocatieType_Wegvak', 'SamengesteldLocatieType_Junctie',
    

(1326104, 43)

### Convert object columns to categorical

1. get column names with object dtype
2. per column, count the amount of unique values
3. create a series and sort them descending

In [6]:
# check memory usage before
df.memory_usage(deep=True).sum() / 1_000_000

2318.292435

In [7]:
object_cols = df.head(1).select_dtypes(include=object).columns
df[object_cols] = df[object_cols].astype("category")

In [8]:
# check memory usage after converting to categorical
df.memory_usage(deep=True).sum() / 1_000_000

161.896126

# Data prep

### Create target variable

In [9]:
# Save target translation for later
target_translation = dict(zip(df['Afloop3'].cat.categories, range(len(df['Afloop3'].cat.categories))))
df['Target'] = df['Afloop3'].cat.codes
df = df.drop(columns="Afloop3")

In [10]:
df.head()

,Wegverlichting,ProvincieNaam,EndpuntAfstand,Lichtgesteldheid,WegBeheerder,Latitude,WegNummer,VoertuigVerlichting,AangrijppuntType,Aangrijppunt1,BeginpuntAfstand,AantalWegvakkenMetJunctie,KaderwetgebiedNaam,Volgnummer,ActueellWegvakkenMetJunctie,WegdeelLetter,ObjectType,MassaLeegVoertuig,AantalPartijen,RijbewijsCategry,AangegevenMaxSnelheid,Wegsituatie,BebouwdeKom,Aard,Inrichting,Wegdekken,Baansubsoort,Schade,APKGekeurd,StraatType,VastgelegdePlaats1,Wegverharding,VoertuigVerzekerd,KlokAanduingAlsEnd,RelatievePositie,Wegvak_BeginDat,Beweging1,RibewijsGeldg,VoorgenomenBeweging,GekoppledNiveau,Weergesteldheid1,KlokAanduingAlsBegin,Target
0,NaN,Zeeland,NaN,Daglicht,G,51.313763,NaN,Niet aanwezig/nvt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Overig vast object,NaN,2,NaN,30.0,Rechte weg,Binnen,Vast voorwerp,NaN,Droog,NaN,NaN,NaN,PTT-straat,Trottoir/berm,Klinkers,NaN,10.0,NaN,20030501.0,NaN,NaN,NaN,Ongeval gekoppeld op straat niveau,Droog,3.0,2
1,Niet aanwezig,Noord-Brabant,NaN,Daglicht,G,51.527046,NaN,Onbekend,V,Rechterflank,NaN,NaN,REGIO NOORD BRABANT-OV,1.0,NaN,NaN,Motor,210.0,1,A1,80.0,Bocht,Buiten,Eenzijdig,geen zijspan toegestaan,Droog,NaN,J,NaN,PTT-straat,Rijbaan,Overig asfalt,J,10.0,NaN,19980216.0,Stilstand,J,Vooruit,Ongeval exact gekoppeld aan BN,Droog,4.0,1
2,Niet brandend,Zeeland,NaN,Daglicht,G,51.499148,NaN,Onbekend,V,Rechts voor,NaN,NaN,NaN,NaN,NaN,NaN,Personenauto,1405.0,2,B,50.0,Rechte weg,Binnen,Kop/staart,combinatiewagen,Droog,NaN,J,J,PTT-straat,Trottoir/berm,Overig asfalt,J,8.0,NaN,20030201.0,NaN,J,Achteruit,Ongeval exact gekoppeld aan BN,Droog,2.0,2
3,NaN,Zeeland,NaN,Daglicht,G,51.313763,NaN,Onbekend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Onbekend voertuig i.g.v. doorrijder,NaN,2,NaN,50.0,Rechte weg,Binnen,Vast voorwerp,NaN,NaN,NaN,NaN,NaN,PTT-straat,NaN,Overig asfalt,NaN,10.0,NaN,20030501.0,NaN,NaN,NaN,Ongeval gekoppeld op straat niveau,Onbekend,3.0,2
4,Brandend,Zeeland,NaN,Daglicht,G,51.313218,NaN,Niet aanwezig/nvt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Overig vast object,NaN,2,NaN,50.0,Rechte weg,Buiten,Vast voorwerp,NaN,Nat,NaN,NaN,NaN,PTT-straat,Trottoir/berm,Beton,NaN,7.0,NaN,20030501.0,NaN,NaN,NaN,Ongeval exact gekoppeld aan BN,NaN,12.0,2
